In [1]:
import psycopg2

In [2]:
# 数据库连接参数
dbname = ""
user = ""
password = ""
host = ""  # 或者是你 Docker 容器的 IP 地址，如果你在不同的机器上
port = ""

# 连接字符串
conn_string = f"host={host} dbname={dbname} user={user} password={password} port={port}"
# 连接到数据库
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [3]:
from psycopg2 import Error as Psycopg2Error

def execute_query(query):
    # 数据库连接参数
    dbname = ""
    user = ""
    password = ""
    host = ""  # 或者是你 Docker 容器的 IP 地址，如果你在不同的机器上
    port = ""
    
    # 连接字符串
    conn_string = f"host={host} dbname={dbname} user={user} password={password} port={port}"
    # 连接到数据库
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()
    try:
        # 使用传入的 cursor 执行 SQL 查询
        cur.execute(query)
        # 获取查询结果
        results = cur.fetchall()
        return results
    except Psycopg2Error as e:
        # 这里可以打印异常信息，方便调试
        print(e)
        return "error"
    finally:
        # 关闭 cursor 和连接
        cur.close()
        conn.close()


In [4]:
res = execute_query("SELECT COUNT(*) FROM tb_process WHERE create_time BETWEEN '2023-06-01' AND '2023-06-15';")
print(res)

[(21238,)]


In [6]:
gold = []
pred = []
err_gold = 0
err_pred = 0

err_flag_gold = [] # 1是error，0不是
err_flag_pred = []

with open("./gold.txt","r",encoding="utf-8") as f:
    for line in f.readlines():
        gold.append(line.strip())
with open("./pred.txt","r",encoding="utf-8") as f:
    for line in f.readlines():
        pred.append(line.strip())

equal = 0
for idx,(g,p) in enumerate(zip(gold,pred)):
    # print("序号：",idx)
    # print("##真实sql:##",)
    res_g = execute_query(g)
         
    # print("##预测sql:##")
    res_p = execute_query(p)
    
    if res_g != res_p:
        print("序号：",idx)
        print(g)
        print(p)
    if res_g == 'error':
        err_flag_gold.append(1)
    else:
        err_flag_gold.append(0)

    if res_p == 'error':
        err_flag_pred.append(1)
    else:
        err_flag_pred.append(0)


    if res_g!='error':
        equal += (res_g == res_p)
    if res_g == 'error':
        err_gold += 1
    if res_p == 'error':
        err_pred += 1

print("error gold:",err_gold)
print("error pred:",err_pred)
print("correct:",equal)

no results to fetch
no results to fetch
column "creator" does not exist
LINE 1: ...CT COUNT(*) AS total FROM tb_product_system WHERE creator = ...
                                                             ^

序号： 21
select count(*) as total from tb_product_system p where create_user = '10043' and deleted = 0
SELECT COUNT(*) AS total FROM tb_product_system WHERE creator = '10043' AND deleted = '0'
序号： 33
SELECT * FROM tb_process_doc WHERE valid_from > '2022-01-01';
SELECT * FROM tb_process_doc WHERE valid_from > '2020-01-01'
序号： 62
SELECT * FROM tb_process WHERE create_time BETWEEN '2023-06-01' AND '2023-06-15';
SELECT * FROM tb_process WHERE create_time >= '2022-01-01' AND create_time <= '2023-01-01'
error gold: 1
error pred: 2
correct: 62


In [7]:
equal/(len(gold)-err_gold)

0.9538461538461539

In [6]:
len(gold)

66

In [19]:
len(pred)

89

In [20]:
print(sum(err_flag_gold))
print(sum(err_flag_pred))

42
47


In [15]:
s = "select id,username,password,real_name,nickname,id_number,avatar,gender,email,mobile,user_type,org_id,super_admin,person_auth_id,enterprise_auth_id,status,remark,open_id,last_login_time,valid_time,creator,create_time,updater,update_time,version,deleted from sys_user where deleted=0"
res = execute_query(s)

column "valid_time" does not exist
LINE 1: ...ise_auth_id,status,remark,open_id,last_login_time,valid_time...
                                                             ^
